pre-trained MobileNetV2 as the encoder + UNet decoder
Import libraries

In [1]:
import os
import numpy as np
import cv2
from glob import glob
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import Conv2D, Activation, BatchNormalization
from tensorflow.keras.layers import UpSampling2D, Input, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.metrics import Recall, Precision, MeanIoU
from tensorflow.keras import backend as K

In [3]:
np.random.seed(0)
tf.random.set_seed(0)

read one sample from image and mask to get the size of pics

In [4]:
IMAGE_SIZE = 512
EPOCHS = 50
BATCH = 8
LR = 1e-4
PATH = "data/"

Load and split data into train, val, test

In [5]:
def load_data(path, split=0.1):
    images = sorted(glob(os.path.join(path, "image/*")))
    masks = sorted(glob(os.path.join(path, "mask/*")))

    total_size = len(images)
    valid_size = int(split * total_size)
    test_size = int(split * total_size)

    train_x, valid_x = train_test_split(images, test_size=valid_size, random_state=0)
    train_y, valid_y = train_test_split(masks, test_size=valid_size, random_state=0)

    train_x, test_x = train_test_split(train_x, test_size=test_size, random_state=0)
    train_y, test_y = train_test_split(train_y, test_size=test_size, random_state=0)

    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)

read images and masks, resize to 256 * 256 and normalize. The original is 224*224*3

In [6]:
def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (IMAGE_SIZE, IMAGE_SIZE))
    x = x/255.0
    return x

def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (IMAGE_SIZE, IMAGE_SIZE))
    x = x/255.0
    x = np.expand_dims(x, axis=-1)
    return x

The tow function below help to build data pipeline

In [7]:
def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float64, tf.float64])
    x.set_shape([IMAGE_SIZE, IMAGE_SIZE, 3])
    y.set_shape([IMAGE_SIZE, IMAGE_SIZE, 1])
    return x, y

def tf_dataset(x, y, batch=8):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.repeat()
    return dataset

The model function builds the architecture for encoder+decoder

In [24]:
def model():
    inputs = Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3), name="input_image")
    
    encoder = MobileNetV2(input_tensor=inputs, weights="imagenet", include_top=False, alpha=1.0)
    skip_connection_names = ["input_image", "block_1_expand_relu", "block_3_expand_relu", "block_6_expand_relu"]
    encoder_output = encoder.get_layer("block_13_expand_relu").output
    
    f = [16, 32, 48, 64]
    x = encoder_output
    for i in range(1, len(skip_connection_names)+1, 1):
        x_skip = encoder.get_layer(skip_connection_names[-i]).output
        x = UpSampling2D((2, 2))(x)
        x = Concatenate()([x, x_skip])
        
        x = Conv2D(f[-i], (3, 3), padding="same")(x)
        x = BatchNormalization()(x)
        x = Activation("relu")(x)
        
        x = Conv2D(f[-i], (3, 3), padding="same")(x)
        x = BatchNormalization()(x)
        x = Activation("relu")(x)
        
    x = Conv2D(1, (1, 1), padding="same")(x)
    x = Activation("sigmoid")(x)
    
    model = Model(inputs, x)
    return model

Dice coefficient measuers performance

In [25]:
smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

In [26]:
(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(PATH, split=0.1)
train_dataset = tf_dataset(train_x, train_y, batch=BATCH)
valid_dataset = tf_dataset(valid_x, valid_y, batch=BATCH)

Train the model, adam optimizer with dice coefficient loss.

In [27]:
model = model()
opt = tf.keras.optimizers.Adam(LR)
metrics = [MeanIoU(num_classes=2), Recall(), Precision()]
model.compile(loss=dice_loss, optimizer=opt, metrics=metrics)

/Users/junh/opt/anaconda3/envs/tf/lib/python3.7/site-packages/keras_applications/mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


9412608/9406464 [==============================] - 1s 0us/step


Two callbacks help to train the model

In [28]:
callbacks = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=6),
    EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
]

In [29]:
train_steps = len(train_x)//BATCH
valid_steps = len(valid_x)//BATCH

if len(train_x) % BATCH != 0:
    train_steps += 1
if len(valid_x) % BATCH != 0:
    valid_steps += 1

In [30]:
model.fit(
    train_dataset,
    epochs=EPOCHS,
    steps_per_epoch=train_steps,
    validation_steps=valid_steps,
    callbacks=callbacks,
    validation_data=valid_dataset
)

Train for 24 steps, validate for 3 steps
Epoch 1/50
24/24 [==============================] - 308s 13s/step - loss: 0.6688 - mean_io_u_1: 0.3944 - recall_1: 0.5327 - precision_1: 0.3251 - val_loss: 0.7556 - val_mean_io_u_1: 0.4027 - val_recall_1: 2.4094e-04 - val_precision_1: 0.1023
Epoch 2/50
24/24 [==============================] - 295s 12s/step - loss: 0.5465 - mean_io_u_1: 0.3944 - recall_1: 0.9169 - precision_1: 0.4672 - val_loss: 0.7015 - val_mean_io_u_1: 0.4027 - val_recall_1: 0.5553 - val_precision_1: 0.2633
Epoch 3/50
24/24 [==============================] - 294s 12s/step - loss: 0.4481 - mean_io_u_1: 0.3944 - recall_1: 0.9557 - precision_1: 0.6231 - val_loss: 0.6802 - val_mean_io_u_1: 0.4027 - val_recall_1: 0.9992 - val_precision_1: 0.2413
Epoch 4/50
24/24 [==============================] - 294s 12s/step - loss: 0.3695 - mean_io_u_1: 0.3944 - recall_1: 0.9663 - precision_1: 0.7401 - val_loss: 0.6173 - val_mean_io_u_1: 0.4027 - val_recall_1: 0.9992 - val_precision_1: 0.2420
Epo

Evaluate the trained model on the test dataset

In [19]:
test_dataset = tf_dataset(test_x, test_y, batch=BATCH)

test_steps = (len(test_x)//BATCH)
if len(test_x) % BATCH != 0:
    test_steps += 1

model.evaluate(test_dataset, steps=test_steps)

3/3 [==============================] - 9s 3s/step - loss: 0.4508 - mean_io_u: 0.3870 - recall: 0.9871 - precision: 0.5052


[0.45082863171895343, 0.38695064, 0.98707616, 0.5051604]

In [ ]:
model.save("src/models1.h5")

In [ ]:
model = tf.keras.models.load_model('models.h5',custom_objects={'dice_loss': dice_loss, 'dice_coef':dice_coef})

In [ ]:
def read_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = cv2.resize(x, (IMAGE_SIZE, IMAGE_SIZE))
    x = x/255.0
    return x

def read_mask(path):
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (IMAGE_SIZE, IMAGE_SIZE))
    x = np.expand_dims(x, axis=-1)
    x = x/255.0
    return x

In [ ]:
model.predict(np.expand_dims(read_image(train_x[0]), axis=0))[0]>0.5